In [14]:
import pandas as pd
import numpy as np
import pickle
from collections import Counter, defaultdict

In [15]:
import sys
import os
def add_sys_path(p):
    p = os.path.abspath(p)
    if p not in sys.path:
        sys.path.append(p)

add_sys_path('..')

In [16]:
import evaluate

In [17]:
import data_split

In [18]:
from evaluate import read_dataset

In [19]:
import json

In [20]:
from tqdm.auto import tqdm, trange

# Чтение и сплит данных

In [21]:
v_ds = read_dataset('../data/training_data/training_verbs.tsv',  lambda x: json.loads(x))

In [22]:
v_ds['АМНИСТИРОВАТЬ']

[['115444-V', '120034-V'], ['117915-V', '135001-V']]

In [24]:
train, dev, test1, test2, hid, forbidden_words = data_split.split_dict(v_ds)

# Применение бейзлайна

In [265]:
import my_knn
from importlib import reload
reload(my_knn)

<module 'my_knn' from 'C:\\Users\\ddale\\YandexDisk\\code\\NLP\\taxonomy-enrichment\\experiments\\my_knn.py'>

In [163]:
import gensim
# this is araneum 2018 ft model from rusvectores
ft = gensim.models.fasttext.FastTextKeyedVectors.load(
    'C:/Users/ddale/Downloads/NLP/rusvectores/model.model'
)

In [266]:
#embedder = my_knn.SentenceEmbedder(ft=ft, n=300, normalize_word=True)
embedder = my_knn.SentenceEmbedder(ft=ft, n=300, normalize_word=True, pos_weights={'INFN': 1.0, 'PREP': 0.1})
print(embedder('привет как дела').shape)

(300,)


In [132]:
taiga = gensim.models.KeyedVectors.load_word2vec_format(
    'C:/Users/ddale/Downloads/NLP/rusvectores/taiga_skipgram/model.bin', 
    binary=True,
)

In [267]:
wrapper = my_knn.W2VWrapper(taiga)

In [165]:
verbs_storage, verbs_rel_storage, rel_df = my_knn.prepare_storages(
    synsets_filename='../data/ruwordnet/synsets.V.xml',
    relations_filename='../data/ruwordnet/synset_relations.V.xml',
    forbidden_words=forbidden_words
)

number of texts: 32822
forbidden senses are 1390
numer of ids 7521 long list is 28657



5752
186
5
7480


In [166]:
len(verbs_storage.texts_long)

28657

In [167]:
import random

In [168]:
from pymorphy2 import MorphAnalyzer

morphAnalyzer = MorphAnalyzer()

def word2pos(w):
    parses = morphAnalyzer.parse(w)
    if not parses:
        return None
    parse = parses[0]
    if not parse.tag:
        return None
    if not parse.tag.POS:
        return None
    return parse.tag.POS

In [170]:
s = random.sample(verbs_storage.texts_long, 5)
for t in s:
    print(t)
    print([word2pos(w) for w in my_knn.tokenize(t)])

РАЗУЧИТЬСЯ, УТРАТИТЬ УМЕНИЕ
['INFN', 'INFN', 'NOUN']
ПОРТРЕТИРОВАТЬ
['INFN']
ПОДПУСКАТЬ ; ПОДПУСКАТЬ БЛИЗКИЙ ; ПОДПУСКАТЬ НА БЛИЗКИЙ РАССТОЯНИЕ ; ПОДПУСТИТЬ ; ПОДПУСТИТЬ БЛИЖЕ ; ПОДПУСТИТЬ БЛИЗКИЙ ; ПОДПУСТИТЬ НА БЛИЗКИЙ РАССТОЯНИЕ
['INFN', 'INFN', 'ADJF', 'INFN', 'PREP', 'ADJF', 'NOUN', 'INFN', 'INFN', 'COMP', 'INFN', 'ADJF', 'INFN', 'PREP', 'ADJF', 'NOUN']
ШАРОМЫЖНИЧАТЬ
['INFN']
РАЗРАБАТЫВАТЬ МЕСТОРОЖДЕНИЕ ГАЗ
['INFN', 'NOUN', 'NOUN']


In [171]:
from sklearn.neighbors import KDTree
vecs = np.stack([embedder(t) for t in tqdm(verbs_storage.texts_long) ])
tree = KDTree(vecs)

In [271]:
w2v_vecs = np.stack([wrapper(t) for t in tqdm(verbs_storage.texts_long) ])
w2v_tree = KDTree(w2v_vecs)

In [173]:
# проверка, что сравнение само с собой всегда получает скор 1
dev_pred = {k: [v0 for val in vals for v0 in val] for k, vals in dev.items()}
mean_ap, mean_rr = evaluate.get_score(dev, dev_pred, k=10)
print(mean_ap, mean_rr)

1.0 1.0


In [276]:
def w2v_scorer(text1, text2):
    return np.dot(wrapper.get_text_vec(text1), wrapper.get_text_vec(text2)) ** 2

In [277]:
#texts = ttest_test1.one_text.drop_duplicates()
small_hypos = {}
for txt in tqdm(dev.keys()):
    hypos = my_knn.hypotheses_knn(
        txt, 
        #index=tree, text2vec=embedder, 
        index=w2v_tree, text2vec=wrapper,
        synset_storage=verbs_storage, rel_storage=verbs_rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        return_hypotheses=True,
        neighbor_scorer=w2v_scorer,
    )
    hypos = my_knn.transitivity_rerank(hypos, txt, verbs_storage.id2synset, mul=1.001)
    small_hypos[txt] = my_knn.get_top(hypos, 10)

Когда я выкинул из синсетов запрещённое, получаю 21% 27%, а если ещё из отношений, то 20% 26%. 

Хотя на паблик тесте вообще 24% 27%. 

Вывод: вообще-то перед сабмишнами надо пересчитываться на полной выборке. 

In [278]:
mean_ap, mean_rr = evaluate.get_score(dev, small_hypos, k=10)
print(mean_ap, mean_rr)

0.260688984766166 0.3427239799723022


```
0.20069866126913108 0.26436294875892197 - baseline
0.19999533929903057 0.2675482049643124 - not normalize word vecs (inconclusive)

0.18938824970704168 0.24604239906253328 - baseline + weight{verb = 3}
0.19966265402507014 0.2668610844785341 - baseline + weight{verb=1.1}
0.20093569120414756 0.2669250026632577 - baseline + weight{prep=0.8} (definitely helps!)
0.20307894783565925 0.2682939171194206 - baseline + weight{prep=0.1} (we'll keep it this way)
0.2034944160363623 0.26912485352082666 - transitivity rerank (1.01)
0.2032758069670821 0.2687786300202407  - transitivity rerank (1.03)
0.1968120805369127 0.26168903803132    - transitivity rerank (1.2)

0.22727974858847338 0.2913257696814745 - penalize by w2v similarity ^ 1
0.23582614253755194 0.3011691701289017 - similarity ^ 2 
0.23904202620645573 0.3038670501757751- similarity ^ 3
0.23729981179645612 0.3064717162032599 - similarity ^ 5
0.21469851922872057 0.2766565462874188 - exp(1*similarity)
0.23159422605731322 0.2943858527751146 - exp(3*similarity)
0.23811210539398467 0.3061760945989136 - exp(10*similarity)
0.21968147437946095 0.2850990731863215 - exp(30*similarity)
0.26026175384396866 0.3428811121764142*- use w2v only for retrieval (penalty is similarity ** 3) 
0.260688984766166 0.3427239799723022   - use w2v only for retrieval (penalty is similarity ** 2)
```

#### Проверяем полноту моей системы

In [664]:
small_hypos_large = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        index=tree, text2vec=embedder, synset_storage=verbs_storage, rel_storage=verb_rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        result_size=100
    )  
    for txt in tqdm(dev.keys())
}

In [665]:
dev[k]

[['117993-V', '118191-V'], ['107011-V', '108870-V']]

In [666]:
for k, v in small_hypos_large.items():
    gts = {sense for senses in dev[k] for sense in senses}
    small_hypos_large[k] = [sense for sense in v if sense in gts]

In [667]:
print(*evaluate.get_score(dev, small_hypos_large, k=10))

0.623042505592841 0.7449664429530202


Если бы я имел возможность более правильно переранжировать мои ВСЕ кандидаты, я бы получил скор 63%/75%, достаточно, чтобы победить в этом соревновании. Но увы. 

30 кандидатов - 48% / 61%, достаточно высоко тоже. 

10 кандидатов - 36% / 48%, если только выкинуть мусор. Но как это сделать?

### частные случаи

Проверяю, что может быть не так

In [695]:
word = random.choice(list(dev.keys()))
print(word)

print(dev[word])
print([
    [verbs_storage.id2synset[id]['@ruthes_name'] for id in ids]
    for ids in dev[word]
])

ОБЕСТОЧИТЬСЯ
[['107417-V', '111842-V', '137256-V']]
[['ОТДЕЛИТЬ ЧАСТЬ ОТ ЦЕЛОГО', 'ВЫКЛЮЧИТЬ (ПРЕКРАТИТЬ)', 'ОТКЛЮЧИТЬ ОТ СЕТИ']]


In [696]:
for h in small_hypos[word]:
    indices = [i for i, senses in enumerate(dev[word]) if h in senses]
    print(h, indices, verbs_storage.id2synset[h]['@ruthes_name'])

116921-V [] ПЕРЕЖИТЬ, ИСПЫТАТЬ
120955-V [] ИЗМЕНЕНИЕ СОСТОЯНИЯ
106629-V [] ИЗМЕНИТЬСЯ, ИЗМЕНЕНИЕ
119625-V [] СЛАБЕТЬ, ТЕРЯТЬ СИЛУ
116929-V [] СТРАДАТЬ, ИСПЫТЫВАТЬ СТРАДАНИЯ
111944-V [] НАЗНАЧИТЬ, УСТАНОВИТЬ, ОПРЕДЕЛИТЬ
111086-V [] ГНУТЬСЯ, СОГНУТЬСЯ
144428-V [] ТОКАРНЫЕ РАБОТЫ
112080-V [] СРЕЗАТЬ С КРАЕВ
130868-V [] ЗЛЕТЬ, ОЗЛОБИТЬСЯ


In [681]:
print(evaluate.compute_ap(dev[word], small_hypos[word], 10))
print(evaluate.compute_rr([j for i in dev[word] for j in i], small_hypos[word], 10))

0.0
0.0


In [578]:
tag = morphAnalyzer.parse('корова')[0].tag
print(tag)
print(tag.transitivity)

NOUN,anim,femn sing,nomn
None


In [584]:
hypos = my_knn.hypotheses_knn(
        word, 
        index=tree, text2vec=embedder, synset_storage=verbs_storage, rel_storage=verb_rel_storage,
        decay=3,  k=100, grand_mult=0.5,
        return_hypotheses=True
    ) 

In [623]:
new_hypos = transitivity_rerank(hypos, word, verbs_storage.id2synset, mul=2)

In [627]:
new_top = get_top(new_hypos)

In [628]:
for h in new_top:
    indices = [i for i, senses in enumerate(dev[word]) if h in senses]
    print(h, indices, verbs_storage.id2synset[h]['@ruthes_name'])

119562-V [] ПРЕГРАДИТЬ, МЕШАТЬ ДВИЖЕНИЮ
110719-V [3] ЛИТЬ ЖИДКОСТЬ
111098-V [] ПЛАВЛЕНИЕ ВЕЩЕСТВА
116248-V [] ПЕРЕГОРОДИТЬ
144157-V [] ОЧИСТКА ОТ ВРЕДНЫХ ВЕЩЕСТВ
128431-V [] ОБЕЗВРЕДИТЬ
106483-V [] УНИЧТОЖИТЬ, ПРЕКРАТИТЬ СУЩЕСТВОВАНИЕ
4255-V [] ОХРАНА ВОД
114638-V [] СМОЧИТЬ
118060-V [] РАЗМЫТЬ ПОТОКОМ


In [1]:
import gensim

In [149]:
import random

In [31]:
from pymorphy2 import MorphAnalyzer

morphAnalyzer = MorphAnalyzer()

In [ ]:
prefix2word = defaultdict(set)
for w in taiga.vocab.keys():
    for n in range(1, len(w)-2):
        prefix2word[w[:-n]].add(w)
prefix2word = {k: v for k, v in prefix2wordx2word.items()}

In [105]:
POS_MAP = {
    'INFN': 'VERB',
    'ADJF': 'ADJ',
    'ADVB': 'ADV',
}
POS_MISS = {
    'PREP'
}

In [94]:
def find_prefix(word, mapping, min_len=2):
    if word in mapping:
        return mapping[word]
    for i in range(1, len(word)-min_len):
        t = mapping.get(word[:-i])
        if t is not None:
            return t

In [124]:
synset_id = random.choice(list(verbs_storage.id2synset.keys()))
text = verbs_storage.id2synset[synset_id]['@ruthes_name']
print(text)

ХРИСТИАНИЗАЦИЯ


In [125]:
toks = my_knn.tokenize(text)
for tok in toks:
    parse = morphAnalyzer.parse(tok)[0]
    tag = parse.tag
    print(tok, tag.POS)
    new_pos = POS_MAP.get(tag.POS, tag.POS)
    if new_pos in POS_MISS:
        continue
    key = tok + '_' + new_pos
    key2 = parse.normal_form + '_' + new_pos
    if key in taiga.vocab:
        print('yes')
    elif key2 in taiga.vocab:
        print('nf')
    else:
        print(find_prefix(key, prefix2word))

христианизация NOUN
yes


In [127]:
taiga.vectors.shape[1]

300

In [56]:
prefix2word['завин']

{'завинчивать_VERB', 'завинчиваться_VERB', 'завинщать_VERB'}

In [45]:
taiga.most_similar_cosmul('завинтить_VERB')

KeyError: "word 'завинтить_VERB' not in vocabulary"

In [42]:
taiga['завинить_VERB']

KeyError: "word 'завинить_VERB' not in vocabulary"

In [ ]:
def 

In [ ]:
taiga.mo

In [697]:
for h in my_knn.hypotheses_knn(
        word, 
        index=tree, text2vec=embedder, synset_storage=verbs_storage, rel_storage=verb_rel_storage,
        decay=3,  k=100, grand_mult=0.5,
        verbose=1,
    ) :
    print(h, verbs_storage.id2synset[h]['@ruthes_name'])

0.7517580250040728 1 129220-V СТОЧИТЬ 2
0.7564952747742115 1 129342-V УЖЕСТОЧИТЬСЯ 1
0.7851116630549265 1 117811-V ОЖЕСТОЧИТЬСЯ 1
0.7935909973567986 1 129220-V СТАЧИВАТЬ ; СТОЧИТЬ ; СТОЧИТЬ (СРЕЗАТЬ) 2
0.8237797165360515 1 129342-V ЗАВИНТИТЬ ГАЙКА ; ЗАВИНЧИВАТЬ ГАЙКА ; УЖЕСТОЧАТЬ ; УЖЕСТОЧАТЬСЯ ; УЖЕСТОЧИТЬ ; УЖЕСТОЧИТЬ (СДЕЛАТЬ ЖЕСТКИМ, ЖЕСТЧЕ) ; УЖЕСТОЧИТЬСЯ 1
0.8440579620242072 1 129220-V СТОЧИТЬ (СРЕЗАТЬ) 2
0.8484946725978589 1 129342-V УЖЕСТОЧИТЬ 1
0.8516737881101539 1 117811-V ОЖЕСТОЧАТЬСЯ ; ОЖЕСТОЧИТЬСЯ 1
0.9125355716624214 1 131003-V КРЮЧИТЬСЯ 1
0.922303187490691 1 107104-V РАЗОРАТЬСЯ 1
0.9236072398647539 1 116929-V ПРОМУЧИТЬСЯ 1
0.9240812043003783 1 117325-V ПРОМАЯТЬСЯ 1
0.924514512359255 1 116929-V ЗАМУЧИТЬСЯ 1
0.9245191921964133 1 118993-V ОТОЩАТЬ 1
0.9249648742502856 1 128589-V РАЗБАЛИВАТЬСЯ ; РАЗБОЛЕТЬСЯ ; РАЗБОЛЕТЬСЯ, РАСХВОРАТЬСЯ ; РАСХВАРЫВАТЬСЯ ; РАСХВОРАТЬСЯ 1
0.9258115948383597 1 125830-V ВЫМОЧИТЬСЯ 2
0.9276494504329605 1 135453-V ЗАТИХАТЬ ; ЗАТИХНУТЬ ; ПОУТИХНУТЬ ; 

Заметные проблемы:
* композиционность ("вывести из эксплуатации", "пробиться через препятствие", "давать на лапу") - ???
* не основной смысл ("блестеть") - нужна диверсификация ответа по смыслам?
* редкий вид слова ("обмолачивать - обмолотить") - добавлять в кандидатов разные формы
* неверно определены валентности, в частности, переходность ("морить голодом" - "страдать")
* слишком далёкие синонимы ("мародерствовать" - "изуверствовать", "паникерствовать", "зверствовать")

In [448]:
for sense in verbs_storage.word2sense[word]:
    print(sense, verbs_storage.id2synset[sense]['@ruthes_name'])

114124-V ОБМОЛОТ ЗЕРНА


# Оценка применения

In [241]:
full_syn_storage, full_rel_storage, full_rel_df = my_knn.prepare_storages(
    synsets_filename='../data/ruwordnet/synsets.V.xml',
    relations_filename='../data/ruwordnet/synset_relations.V.xml',
    forbidden_words=set()
)

number of texts: 32822
forbidden senses are 0
numer of ids 7521 long list is 49002



7408
218
5
10317


In [243]:
embedder = my_knn.SentenceEmbedder(
    ft=ft, n=300, normalize_word=True, pos_weights={'INFN': 1.0, 'PREP': 0.1}, default_weight=0.5
)

In [244]:
full_vecs = np.stack([embedder(t) for t in tqdm(full_syn_storage.texts_long) ])
full_tree = KDTree(full_vecs)

In [279]:
w2v_wrapper = my_knn.W2VWrapper(taiga, pos_weights={'INFN': 1.0, 'PREP': 0.1}, default_weight=0.5, n=300)
full_w2v_vecs = np.stack([w2v_wrapper(t) for t in tqdm(full_syn_storage.texts_long) ])
full_w2v_tree = KDTree(full_w2v_vecs)

In [280]:
public_test_verbs = pd.read_csv('../data/public_test/verbs_public.tsv', header=None)
public_test_verbs.columns = ['text']

In [281]:
def w2v_scorer(text1, text2):
    return np.dot(wrapper.get_text_vec(text1), wrapper.get_text_vec(text2)) ** 3

In [282]:
public_test_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        #index=full_tree, text2vec=embedder, 
        index=full_w2v_tree, text2vec=w2v_wrapper,
        synset_storage=full_syn_storage, rel_storage=full_rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
    )  
    for txt in tqdm(public_test_verbs.text)
}

In [283]:
sub = my_knn.dict2submission(public_test_hypos, full_syn_storage.id2synset)
sub.head(15)

,noun,result,result_text
0,АБСОЛЮТИЗИРОВАТЬ,116390-V,"ИСКАЗИТЬ, ПРЕДСТАВИТЬ В ЛОЖНОМ СВЕТЕ"
1,АБСОЛЮТИЗИРОВАТЬ,141697-V,ПРЕДСТАВИТЬ В ВИДЕ
2,АБСОЛЮТИЗИРОВАТЬ,149806-V,"УТВЕРЖДАТЬ, ДЕЛАТЬ УТВЕРЖДЕНИЕ"
3,АБСОЛЮТИЗИРОВАТЬ,106877-V,ВЫСКАЗАТЬ МНЕНИЕ
4,АБСОЛЮТИЗИРОВАТЬ,119125-V,"РАСПРОСТРАНИТЬСЯ, ПОЛУЧИТЬ РАСПРОСТРАНЕНИЕ"
5,АБСОЛЮТИЗИРОВАТЬ,106456-V,ВЫРАЗИТЬ (ПОКАЗАТЬ)
6,АБСОЛЮТИЗИРОВАТЬ,118698-V,"ДЕЙСТВИЕ, ЦЕЛЕНАПРАВЛЕННОЕ ДЕЙСТВИЕ"
7,АБСОЛЮТИЗИРОВАТЬ,106710-V,"ОПРЕДЕЛИТЬ, ВЫЯСНИТЬ"
8,АБСОЛЮТИЗИРОВАТЬ,117532-V,НАСТОЯТЬ НА СВОЕМ МНЕНИИ
9,АБСОЛЮТИЗИРОВАТЬ,106517-V,"СРАВНИТЬ, СОПОСТАВИТЬ"


In [284]:
sub.to_csv('results/verbs_v5_w2v_only.tsv', sep='\t', encoding='utf-8', header=None, index=None)

Проверка на идентичность с предыдущей заливкой

In [183]:
prev = pd.read_csv('results/verbs_v0_strange.tsv', sep='\t', encoding='utf-8', header=None)
prev.columns = sub.columns

In [191]:
assert prev.shape == sub.shape
assert (prev.noun == sub.noun).all()
assert (prev.result == sub.result).all()

# Финальная заливка

In [286]:
private_test_verbs = pd.read_csv('../data/private_test/verbs_private.tsv', header=None)
private_test_verbs.columns = ['text']

In [287]:
private_test_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        #index=full_tree, text2vec=embedder, 
        index=full_w2v_tree, text2vec=w2v_wrapper,
        synset_storage=full_syn_storage, rel_storage=full_rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
    )  
    for txt in tqdm(private_test_verbs.text)
}

In [288]:
sub = my_knn.dict2submission(private_test_hypos, verbs_storage.id2synset)

In [289]:
sub

,noun,result,result_text
0,АДСОРБИРОВАТЬ,106950-V,"ПРЕВРАТИТЬ (ПРИДАТЬ ИНОЙ ВИД, КАЧЕСТВО)"
1,АДСОРБИРОВАТЬ,114441-V,ОЧИСТИТЬ ОТ ПРИСУТСТВИЯ
2,АДСОРБИРОВАТЬ,111097-V,ИЗМЕНЕНИЕ АГРЕГАТНОГО СОСТОЯНИЯ
3,АДСОРБИРОВАТЬ,106629-V,"ИЗМЕНИТЬСЯ, ИЗМЕНЕНИЕ"
4,АДСОРБИРОВАТЬ,106483-V,"УНИЧТОЖИТЬ, ПРЕКРАТИТЬ СУЩЕСТВОВАНИЕ"
...,...,...,...
3495,ЮМОРИТЬ,116284-V,"ВЫСМЕЯТЬ, ОСМЕЯТЬ"
3496,ЮМОРИТЬ,118698-V,"ДЕЙСТВИЕ, ЦЕЛЕНАПРАВЛЕННОЕ ДЕЙСТВИЕ"
3497,ЮМОРИТЬ,106504-V,ПОСТУПОК ЧЕЛОВЕКА
3498,ЮМОРИТЬ,117970-V,ВАЖНИЧАТЬ


In [290]:
sub.to_csv('results/private_verbs_v5_w2v_only.tsv', sep='\t', encoding='utf-8', header=None, index=None)

# Гипотеза с n-граммами

http://www.ruscorpora.ru/new/corpora-freq.html

In [465]:
ngrams_path = 'C:/Users/ddale/Downloads/NLP/ruscorpora_ngrams'

In [470]:
import os
import pandas as pd

In [473]:
p = ngrams_path

grams_lines = {}
for fn in os.listdir(p):
    if not fn.endswith('txt'):
        continue
    print(fn)
    f0 = int(fn[0])
    with open(os.path.join(p, fn), encoding='utf-8') as f:
        grams_lines[f0] = [l.split('\t') for l in f.readlines()]

1grams-3.txt
2grams-3.txt
3grams-3.txt
4grams-2.txt
5grams-2.txt


In [474]:
def get_ngram(l):
    l = [w.strip() for w in l[1:]]
    return '_'.join([w for w in l if w])

In [475]:
grams_freq = {
    n: {
        get_ngram(l): int(l[0])
        for l in d
    }
    for n, d in grams_lines.items()
}

Из биграм мы узнаём, что "захламленной" может быть комната или квартира. БОльшие n не сообщают ничего

In [484]:
for k, v in grams_freq[2].items():
    if 'захламл' in k:
        print(k, v)

был_захламлен 5
захламленной_комнате 5
в_захламленной 4
в_захламленном 4
в_захламленный 4
захламленную_комнату 4
по_захламленному 4
была_захламлена 3
захламленной_квартире 3
захламленности_и 3


Фасттекст умнее - он знает, что это слово похоже на замусорять, загромождивать и загаживать, что бы это ни значило :)

In [485]:
ft.most_similar('захламлять')

[('захламлать', 0.9291685819625854),
 ('захламливать', 0.9029788970947266),
 ('захламленный', 0.8950067758560181),
 ('захламленность', 0.7632160186767578),
 ('замусорять', 0.6844406127929688),
 ('загромождивать', 0.6693085432052612),
 ('загаживать', 0.6531472206115723),
 ('загромождать', 0.6455291509628296),
 ('замусорить', 0.6451689600944519),
 ('задымлять', 0.642647385597229)]